In [13]:
import pandas as pd
import numpy as np
import time
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import pipeline
from sklearn import impute, compose, metrics, preprocessing
heart_df = pd.read_csv('data/heart.csv')

In [8]:
x = heart_df.iloc[:, :-1]
y = heart_df.iloc[:, -1]

bins = [25, 35, 45, 55, 100]
category = ['young', 'adult', 'old', 'senior']

x['age_grps'] = pd.cut(x['age'], bins, labels=category)
x = x.drop(['age'], axis=1)


In [10]:
x

,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,age_grps
0,1,3,145,233,1,0,150,0,2.3,0,0,1,senior
1,1,2,130,250,0,1,187,0,3.5,0,0,2,adult
2,0,1,130,204,0,0,172,0,1.4,2,0,2,adult
3,1,1,120,236,0,1,178,0,0.8,2,0,2,senior
4,0,0,120,354,0,1,163,1,0.6,2,0,2,senior
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0,0,140,241,0,1,123,1,0.2,1,0,3,senior
299,1,3,110,264,0,1,132,0,1.2,1,0,3,adult
300,1,0,144,193,1,1,141,0,3.4,1,2,3,senior
301,1,0,130,131,0,1,115,1,1.2,1,1,3,senior


In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)


cat_variables = ['age_grps']

# categories = pipeline.Pipeline(steps=[('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')), ('ordinal', preprocessing.())])

# pre_pro = compose.ColumnTransformer(transformers=[('cat', categories, cat_variables)], remainder='drop')

classifiers = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Skl GBM": GradientBoostingClassifier(n_estimators=100),
    "Logistic Regression": LogisticRegression(),
    "SVM": SVC(kernel='linear')
}

# classifiers = {name: pipeline.make_pipeline(pre_pro, model) for name, model in classifiers.items()}

x_train = preprocessing.OrdinalEncoder(categories=cat_variables).fit_transform(x_train)
x_test = preprocessing.OrdinalEncoder(categories=cat_variables).transform(x_test)

x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().transform(x_test)

x_train
#
# results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})
#
# for model_name, model in classifiers.items():
#     start_time = time.time()
#     model.fit(x_train, y_train)
#     pred = model.predict(x_test)
#
#     total_time = time.time() - start_time
#
#     results = results.append({"Model": model_name,
#                               "Accuracy": metrics.accuracy_score(y_test, pred) * 100,
#                               "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred) * 100,
#                               "Time": total_time},
#                              ignore_index=True)
#
# results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
# results_ord.index += 1
# results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

ValueError: Shape mismatch: if categories is an array, it has to be of shape (n_features,).